In [1]:
import petlink32
import numpy as np
import time

In [2]:
#prints from Siemens walkthrough, a bed position tag
ew1 =  0xc40d180c
hbp = 0xd180c
hbp_se = -190452

In [3]:
my_hbp = petlink32.HORIZONTAL_BED_POSITION.evaluate(hbp)
print("hbp diff = {}".format(hbp-my_hbp))

hbp diff = 0


In [4]:
my_hbp_se = petlink32.sign_extend(my_hbp, 20)
print(my_hbp_se)
diff = hbp_se - my_hbp_se
print("hbp_se diff = {}".format(diff))

-190452
hbp_se diff = 0


In [5]:
print(petlink32.sign_extend(np.array([my_hbp]*3), 20))

[-190452 -190452 -190452]
